In [42]:
#importing all needed and required libraries
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import csv
import surprise
from surprise import Reader, Dataset
from surprise import accuracy
from surprise import SVD

#loading training data and test data
train_data= pd.read_csv(r"C:\Users\Pucado R\Downloads\Data Science Nigeria\train.csv")
test_data= pd.read_csv(r"C:\Users\Pucado R\Downloads\Data Science Nigeria\test.csv")

In [43]:
test_data= test_data.drop('Rating', axis=1)

In [44]:
#data encoding, changing joke identifier and viewers id to their numeric values

#defining function to change viewers id to a numeric value
def redefine_viewers_id(dataset):
    dataset['Viewers_ID']= dataset['Viewers_ID'].str.slice(1)
    
redefine_viewers_id(train_data)
redefine_viewers_id(test_data)

In [57]:
#defining function to change joke identifier 
train_jokes= train_data['Joke_identifier'].unique()
test_jokes= test_data['Joke_identifier'].unique()

#combining joke identifiers from test and train set to ensure homogenuity
jokes= np.append(train_jokes, test_jokes)
jokes= np.unique(jokes)

#Using label encoder
le= LabelEncoder()
le.fit(jokes)

#transform to numeric value
def create_joke_id(dataset):
    dataset['Joke_identifier']= le.transform(dataset['Joke_identifier'])
    
create_joke_id(train_data)
create_joke_id(test_data)
test_data

,Viewers_ID,Joke_identifier,Response_ID
0,1,14,A1_Akpororo Lagos 1
1,1,9,A1_Akpororo Abuja 1
2,1,13,A1_Akpororo Abuja 5
3,1,87,A1_I Go Dye Benin 1
4,1,89,A1_I Go Dye Benin 3
5,1,94,A1_I Go Dye Uyo 1
6,1,95,A1_I Go Dye Uyo 2
7,1,131,A1_Seyi Law Abuja 3
8,1,132,A1_Seyi Law Abuja 4
9,1,71,A1_Gordons Lagos 2


In [59]:
#data preparation for train data
reader = Reader(rating_scale=(-5, 5))
data_train = Dataset.load_from_df(train_data[['Viewers_ID', 'Joke_identifier', 'Rating']], reader)
test_data['Rating'] = 0
data_test = Dataset.load_from_df(test_data[['Viewers_ID', 'Joke_identifier', 'Rating']], reader)

In [60]:
#build full trainset and use cross validation for evaluation
trainset= data.build_full_trainset()

In [61]:
# Using the SVD algorithm.
algo = SVD(n_epochs=50, lr_all=0.005, reg_all=0.2)

# Train the algorithm on the trainset
algo.fit(trainset)

In [63]:
#make predictions on testdata
def define_prediction(x):
  pred = algo.predict(x['Viewers_ID'], x['Joke_identifier'])
  return pred.est

test_data['Rating'] = test_data.apply(define_prediction, axis=1)

In [64]:
#create dataframe of predictions
result = test_data[['Response_ID', 'Rating']]

result.to_csv('submission.csv', index=None)